In [1]:
from random import Random
import numpy as np
import pandas as pd
from tva.situation import Situation
from tva.strategies import Strategies
import copy
from tva.schemes import Schemes, VotingScheme
from tva.enums import VotingScheme, Happiness
from tva.voter import Voter

In [2]:
# Create the voting situation
situation = Situation(num_voters=4, num_candidates=4, seed=42)
display(situation)

# Apply the voting scheme to the situation
schemes = Schemes()
voting_scheme = VotingScheme.BORDA
schemes.print_results(situation, verbose=True)



Voter 0: ['A', 'C', 'D', 'B']
Voter 1: ['D', 'A', 'B', 'C']
Voter 2: ['C', 'A', 'D', 'B']
Voter 3: ['A', 'B', 'C', 'D']

Anti plurality: A {'A': 4, 'C': 3, 'D': 3, 'B': 2} 
Two voting: A {'A': 4, 'C': 2, 'D': 1, 'B': 1} 
Borda: A {'A': 10, 'C': 6, 'D': 5, 'B': 3}


In [11]:
situation = Situation(num_voters=4, num_candidates=4)
situation.voters[0].preferences = ["A","B","C","D"]
situation.voters[1].preferences = ["C","B","A","D"]
situation.voters[2].preferences = ["B","A","D","C"]
situation.voters[3].preferences = ["C","B","A","D"]
display(situation)

current_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
print("Borda:", current_winner, scores)

Voter 0: ['A', 'B', 'C', 'D']
Voter 1: ['C', 'B', 'A', 'D']
Voter 2: ['B', 'A', 'D', 'C']
Voter 3: ['C', 'B', 'A', 'D']

Borda: B {'A': 7, 'B': 9, 'C': 7, 'D': 1}


In [15]:
def bullet_vote(situation: Situation, voter_index: int, voting_scheme:VotingScheme, happiness_func:Happiness) -> bool:
    """voting for just one alternative, despite having the option to vote for several"""
    voter: Voter = situation.voters[voter_index]
    # Save the original happiness of this voter
    original_winner:str = schemes.apply_voting_scheme(voting_scheme, situation.voters) # type: ignore
    original_happiness = voter.calculate_happiness(original_winner, happiness_func)
    voters: list[Voter] =  copy.deepcopy(situation.voters)
    
    bullet_preferences:list[list[str]] = get_bullet_preferences(voter.preferences)
    for pref in bullet_preferences:
        # Find the winner of the voting scheme with the new preference
        voters[voter_index].preferences = pref
        # print("Permutation:", original_preferences)
        new_winner:str = schemes.apply_voting_scheme(voting_scheme, voters) # type: ignore
        # Calculate the happiness of the voter with the new preference
        happiness = voter.calculate_happiness(new_winner, happiness_func)
        if happiness > original_happiness:
            return True
    return False
    
def get_bullet_preferences(elements:list[str]):
    """
    Given an array of elements return a list of lists where each list contains only the first element plus one of the other alternatives
    """
    first_element = elements[0]
    return [[first_element, e] for e in elements[1:]]


bullet_vote(situation, 0, VotingScheme.BORDA, Happiness.LINEAR)


False

Create 1000 situations and check for what fraction of them a strategy is good

We do experiments for each voting scheme separately.<br>
TODO: We need to save, what type of strategy has won this time?

In [1]:
strategies = Strategies()
voting_scheme = VotingScheme.BORDA
situation = Situation(num_voters=5, num_candidates=4)
situation

NameError: name 'Strategies' is not defined

In [33]:
strategies = Strategies()
voting_scheme = VotingScheme.BORDA
nr_repetitions = 1000
num_voters = 4

strategy_counter = 0
for i in range(nr_repetitions):
    situation = Situation(num_voters=num_voters, num_candidates=4)
    # Check if at least one voter has a good strategy
    for voter_index in range(num_voters):
        if strategies.is_any_strategy_good(situation, voter_index, voting_scheme):
            strategy_counter += 1
            break
        
print(strategy_counter/nr_repetitions)

0.877


Plurality cannot be affected by strategic voting because that would require a voter to vote for a candidate they prefer less than their favorite candidate.